# Assignment (sample solution) 1: Regression

## 0.1. Dataset

Bike sharing data – look into the dataset description in the README file.

## Step 1: Uploading and inspecting the data

In [ ]:
import pandas as pd
import os

def load_data(bike_path):
    csv_path = os.path.join(bike_path, "bike_hour.csv")
    return pd.read_csv(csv_path)

In [ ]:
bike_data = load_data("bike_sharing/")
bike_data.head()

In [ ]:
bike_data.info()

In [ ]:
bike_data["season"].value_counts()

In [ ]:
bike_data.describe()

In [ ]:
bike_data = bike_data.drop("instant", axis=1)

In [ ]:
%matplotlib inline 
#so that the plot will be displayed in the notebook
import matplotlib.pyplot as plt

bike_data.hist(bins=50, figsize=(20,15))
plt.show()

**Summary**: the data was collected in a very balanced way, so attributes like *season*, *yr*, *weekday*, *mnth*, *hour* are represented evenly. Features like *casual* and *registered* are tail-heavy. Feature *season* is categorical so will require converting to numerical values. No missing values in this dataset.

## Step 2: Splitting the data into training and test sets

In [ ]:
import numpy as np
np.random.seed(42)

def split_train_test(data, test_ratio):    
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return data.iloc[train_indices], data.iloc[test_indices]

train_set, test_set = split_train_test(bike_data, 0.2)
print(len(train_set), "training instances +", len(test_set), "test instances")

In [ ]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(bike_data, test_size=0.2, random_state=42)
print(len(train_set), "training instances +", len(test_set), "test instances")

In [ ]:
bike_data["hr"].hist(bins=48)
plt.show()

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(bike_data, bike_data["hr"]):
    strat_train_set = bike_data.loc[train_index]
    strat_test_set = bike_data.loc[test_index]

In [ ]:
def hourly_proportions(data):
    return data["hr"].value_counts() / len(data)

compare_props = pd.DataFrame({
    "Overall": hourly_proportions(bike_data),
    "Stratified tr": hourly_proportions(strat_train_set),
    "Random tr": hourly_proportions(train_set),
    "Stratified ts": hourly_proportions(strat_test_set),
    "Random ts": hourly_proportions(test_set),
})
compare_props["Rand. tr %error"] = 100 * compare_props["Random tr"] / compare_props["Overall"] - 100
compare_props["Rand. ts %error"] = 100 * compare_props["Random ts"] / compare_props["Overall"] - 100
compare_props["Strat. tr %error"] = 100 * compare_props["Stratified tr"] / compare_props["Overall"] - 100
compare_props["Strat. ts %error"] = 100 * compare_props["Stratified ts"] / compare_props["Overall"] - 100

compare_props.sort_index()

## Step 3: Exploring the attributes

In [ ]:
bike_data = strat_train_set.copy()

In [ ]:
bike_data.plot(kind='scatter', x='mnth', y='temp', alpha=0.5,
            s=bike_data["registered"]/10, label="registered", figsize=(10,7), 
            c=bike_data["cnt"], cmap=plt.get_cmap("jet"), colorbar="True",
            )
plt.legend()

In [ ]:
bike_data.plot(kind='scatter', x='mnth', y='atemp', alpha=0.5,
            s=(bike_data["registered"] + bike_data["casual"])/10, label="all_users", figsize=(10,7), 
            c=bike_data["cnt"], cmap=plt.get_cmap("jet"), colorbar="True",
            )
plt.legend()

In [ ]:
bike_data.plot(kind='scatter', x='mnth', y='hr', alpha=0.5,
            s=(bike_data["registered"] + bike_data["casual"])/10, label="all_users", figsize=(10,7), 
            c=bike_data["cnt"], cmap=plt.get_cmap("jet"), colorbar="True",
            )
plt.legend()

In [ ]:
bike_data.plot(kind='scatter', x='weekday', y='hr', alpha=0.5,
            s=(bike_data["registered"] + bike_data["casual"])/10, label="all_users", figsize=(10,7), 
            c=bike_data["cnt"], cmap=plt.get_cmap("jet"), colorbar="True",
            )
plt.legend()

In [ ]:
bike_data.plot(kind='scatter', x='hum', y='atemp', alpha=0.5,
            s=(bike_data["registered"] + bike_data["casual"])/10, label="all_users", figsize=(10,7), 
            c=bike_data["cnt"], cmap=plt.get_cmap("jet"), colorbar="True",
            )
plt.legend()

In [ ]:
bike_data.plot(kind='scatter', x='weathersit', y='atemp', alpha=0.5,
            s=(bike_data["registered"] + bike_data["casual"])/10, label="all_users", figsize=(10,7), 
            c=bike_data["cnt"], cmap=plt.get_cmap("jet"), colorbar="True",
            )
plt.legend()

In [ ]:
corr_matrix = bike_data.corr()
corr_matrix

In [ ]:
corr_matrix["cnt"].sort_values(ascending=False)

In [ ]:
from pandas.plotting import scatter_matrix

attributes = ["cnt", "registered", "casual", "temp", "atemp", "yr", "mnth", "hum"]
scatter_matrix(bike_data[attributes], figsize=(12,8))

In [ ]:
bike_data.plot(kind="scatter", x="cnt", y="registered", alpha=0.3)

In [ ]:
bike_data.plot(kind="scatter", x="cnt", y="temp", alpha=0.1)

In [ ]:
# bike_data["temp_to_hum"] = bike_data["temp"] / bike_data["hum"]
# bike_data["atemp_to_hum"] = bike_data["atemp"] / bike_data["hum"]

# to avoid division by zero
bike_data["temp_to_hum"] = np.where(bike_data["hum"] == 0, bike_data["temp"] / 0.1, 
                                    bike_data["temp"] / bike_data["hum"])
bike_data["atemp_to_hum"] = np.where(bike_data["hum"] == 0, bike_data["atemp"] / 0.1, 
                                    bike_data["atemp"] / bike_data["hum"])

bike_data["casual"].where(bike_data["casual"] > 0, 0.1, inplace = True)
bike_data["casual_log"] = np.log(bike_data["casual"])

corr_matrix = bike_data.corr()
corr_matrix["cnt"].sort_values(ascending=False)

**Summary**: the scatter plots show that there is a particular ranges of temperature values leading to a higher count of rented bikes (subject to the month, so less over the winter months). The number of rented bikes is higher around 8am and 5-6pm. Regarding the months, the maximum number of rentals fall within March-October interval, regarding the days of the week – the interval between Tuesday and Saturday (surprisingly, not Monday!). There is a particular cluster of humidity-vs-temperature values that result in higher rental counts. There are a number of attributes highly correlated with the number of rented bikes: among them, *registered* and *casual* most strongly positively correlated, *temp* / *atemp*, *hr* and *yr* strongly positively correlated, *hum* strongly negatively correlated. This can be seen in both correlation values and scatter plots. One might consider using log on registered and casual users, and a combined attribute that reflects the relation between temperature and humidity (e.g., something like a ratio of the temperature value to humidity).


## Step 4: Data preparation and transformations for machine learning algorithms

In [ ]:
bike_data = strat_train_set.drop("cnt", axis=1).drop("dteday", axis=1)
bike_labels = strat_train_set["cnt"].copy()

In [ ]:
def add_features(data):
    # add the transformed features that you found useful before
    data["temp_to_hum"] = np.where(data["hum"] == 0, data["temp"] / 0.1, data["temp"] / data["hum"])
    data["atemp_to_hum"] = np.where(data["hum"] == 0, data["atemp"] / 0.1, data["atemp"] / data["hum"])
                                            
    data["casual"].where(data["casual"] > 0, 0.1, inplace = True)
    data["casual_log"] = np.log(data["casual"])

add_features(bike_data)
bike_data.info()

In [ ]:
bike_data.describe()

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
bike_cat_encoded = encoder.fit_transform(bike_data["season"])
bike_cat_encoded

In [ ]:
encoder.classes_

In [ ]:
from sklearn.preprocessing import LabelBinarizer

encoder = LabelBinarizer()
bike_cat_1hot = encoder.fit_transform(bike_data["season"])
bike_cat_1hot

In [ ]:
from sklearn.base import TransformerMixin # TransformerMixin allows you to use fit_transform method

class CustomLabelBinarizer(TransformerMixin):
    def __init__(self, *args, **kwargs):
        self.encoder = LabelBinarizer(*args, **kwargs)
    def fit(self, x, y=0):
        self.encoder.fit(x)
        return self
    def transform(self, x, y=0):
        return self.encoder.transform(x)

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

# Create a class to select numerical or categorical columns 
# since Scikit-Learn doesn't handle DataFrames yet
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion

bike_num = bike_data.drop("season", axis=1)
num_attribs = list(bike_num)
cat_attribs = ["season"]

num_pipeline = Pipeline([
        ('selector', DataFrameSelector(num_attribs)),
        ('std_scaler', StandardScaler()),
    ])

cat_pipeline = Pipeline([
        ('selector', DataFrameSelector(cat_attribs)),
        ('label_binarizer', CustomLabelBinarizer()),
    ])

full_pipeline = FeatureUnion(transformer_list=[
        ("num_pipeline", num_pipeline),
        ("cat_pipeline", cat_pipeline),
    ])

bike_prepared = full_pipeline.fit_transform(bike_data)
print(bike_prepared.shape)
bike_prepared

## Step 5: Implementation, evaluation and fine-tuning of a regression model

In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(bike_prepared, bike_labels)

In [ ]:
some_data = bike_data.iloc[:5]
some_labels = bike_labels.iloc[:5]
# note the use of transform, as you'd like to apply already learned (fitted) transformations to the data
some_data_prepared = full_pipeline.transform(some_data)

print("Predictions:", list(lin_reg.predict(some_data_prepared)))
print("Actual labels:", list(some_labels))

In [ ]:
from sklearn.metrics import mean_squared_error

bike_predictions = lin_reg.predict(bike_prepared)
lin_mse = mean_squared_error(bike_labels, bike_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

model = Pipeline([('poly', PolynomialFeatures(degree=3)),
                  ('linear', LinearRegression())])

model = model.fit(bike_prepared, bike_labels)
bike_predictions = model.predict(bike_prepared)
lin_mse = mean_squared_error(bike_labels, bike_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor()
tree_reg = tree_reg.fit(bike_prepared, bike_labels)
bike_predictions = tree_reg.predict(bike_prepared)
tree_mse = mean_squared_error(bike_labels, bike_predictions)
tree_mse = np.sqrt(tree_mse)
tree_mse

In [ ]:
from sklearn.model_selection import cross_val_score
    
def analyse_cv(model):   
    scores = cross_val_score(model, bike_prepared, bike_labels,
                             scoring = "neg_mean_squared_error", cv=10)

    # cross-validation expects utility function (greater is better)
    # rather than cost function (lower is better), so the scores returned
    # are negative as they are the opposite of MSE
    sqrt_scores = np.sqrt(-scores) 
    print("Scores:", sqrt_scores)
    print("Mean:", sqrt_scores.mean())
    print("Standard deviation:", sqrt_scores.std())
    
analyse_cv(tree_reg)

In [ ]:
analyse_cv(lin_reg)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor()
analyse_cv(forest_reg)

In [ ]:
from sklearn.model_selection import GridSearchCV

# specify the range of hyperparameter values for the grid search to try out 
param_grid = [{'n_estimators': [3, 10, 30], 'max_features': [4, 8, 16, 20]}]

forest_reg = RandomForestRegressor()
grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                          scoring="neg_mean_squared_error")
grid_search.fit(bike_prepared, bike_labels)

grid_search.best_params_

In [ ]:
cv_results = grid_search.cv_results_
for mean_score, params in zip(cv_results["mean_test_score"], cv_results["params"]):
    print(np.sqrt(-mean_score), params)

In [ ]:
feature_importances = grid_search.best_estimator_.feature_importances_

cat_one_hot_attribs = ['FALL', 'SPRING', 'SUMMER', 'WINTER']
attributes = num_attribs + cat_one_hot_attribs
sorted(zip(feature_importances, attributes), reverse=True)

In [ ]:
lin_feature_importances = lin_reg.coef_
sorted(zip(lin_feature_importances, attributes), reverse=True)

In [ ]:
X_test = strat_test_set.drop("cnt", axis=1).drop("dteday", axis=1)
y_test = strat_test_set["cnt"].copy()
add_features(X_test)
X_test_prepared = full_pipeline.transform(X_test)

final_model = grid_search.best_estimator_
final_predictions = final_model.predict(X_test_prepared)
final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)

final_rmse

In [ ]:
final_model = lin_reg
final_predictions = final_model.predict(X_test_prepared)
final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)

final_rmse

**Summary**: unlike the task described in the practical, this task is very well solved with simple linear regression (more attributes express linear correlation with the target value). The best results are achieved with a linear regression rather than more complex models. The most informative features across the models and correlation analysis include the number of registered and casual users, year, working day, temperature and the ratio of temperature (especially atemp value) to humidity.